# Geração das métricas e gráficos para o programa Mandelbrot

In [1]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Define os valores de N e o número de threads que deseja testar

In [2]:
valores_de_N = [100, 200, 400, 800]
num_threads = [1, 2, 4, 8, 12, 16]

### Cria uma lista de listas para armazenar os tempos, speedup, a efiência e a  confiança

In [3]:
times = []
speedup = []
confidences_95 = []
efficiency = []
for _ in valores_de_N:
    times.append([0] * len(num_threads))
    speedup.append([0] * len(num_threads))
    confidences_95.append([0] * len(num_threads))
    efficiency.append([0] * len(num_threads))

### Executa em loop o código do crivo e armazena os valores na lista "times"

In [4]:
for i, N in enumerate(valores_de_N):
    for j, num in enumerate(num_threads):
        execution_times = []  # Lista para armazenar os tempos de 10 execuções
        for k in range(10):  # Realize 10 execuções
            commands = ["/usr/bin/time", "-f", "%e", "./codes/mandelbrotomp.o", str(num)] + [str(N) for _ in range(3)]
            process = subprocess.Popen(commands, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            output = process.communicate()[0]
            output = output.decode('utf-8').strip()
            #print(output)
            execution_times.append(float(output))  # Armazene o tempo em cada execução
        # Calcule a média dos tempos das 10 execuções
        avg_time = sum(execution_times) / 10
        times[i][j] = avg_time
        # 95% de confiança
        std_dev = np.std(execution_times)
        confidence = 2.262 * std_dev / np.sqrt(10)
        confidences_95[i][j] = confidence

KeyboardInterrupt: 

### Calcula o speedup

In [48]:
for i in range(len(valores_de_N)):
    for j in range(len(num_threads)):
        if times[i][j] == 0:
            speedup[i][j] = 0
        else:
            speedup[i][j] = times[i][0] / times[i][j]

### Calcula a efiência

In [49]:
for i in range(len(valores_de_N)):
    for j in range(len(num_threads)):
        if times[i][j] == 0:
            efficiency[i][j] = 0
        else:
            efficiency[i][j] = speedup[i][j] / num_threads[j]

### Gera os gráficos

In [51]:
for i, N in enumerate(valores_de_N):
    # Plota o gráfico de tempos
    plt.figure()
    plt.bar(num_threads, times[i])
    plt.xlabel('Número de threads')
    plt.ylabel('Tempo (s)')
    plt.title(f'Tempo de execução para N = {N}')
    
    # Define os valores do eixo x para [1, 2, 4, 8]
    plt.xticks(num_threads)

    plt.savefig(f'metrics/omp/times/time_{N}.png')

    # Fecha a figura após salvá-la
    plt.close()

    # Plota o gráfico de speedup
    plt.figure()
    plt.bar(num_threads, speedup[i])
    plt.xlabel('Número de threads')
    plt.ylabel('Speedup')
    plt.title(f'Speedup para N = {N}')
    
    # Define os valores do eixo x para [1, 2, 4, 8]
    plt.xticks(num_threads)

    plt.savefig(f'metrics/omp/speedups/speedup_{N}.png')

    # Fecha a figura após salvá-la
    plt.close()

    # Plota o gráfico de eficiência
    plt.figure()
    plt.bar(num_threads, efficiency[i])
    plt.xlabel('Número de threads')
    plt.ylabel('Eficiência')
    plt.title(f'Eficiência para N = {N}')

    # Define os valores do eixo x para [1, 2, 4, 8]
    plt.xticks(num_threads)

    plt.savefig(f'metrics/omp/efficiencies/efficiency_{N}.png')

    # Fecha a figura após salvá-la
    plt.close()

    # Plota o gráfico de confiança
    plt.figure()
    plt.bar(num_threads, confidences_95[i])
    plt.xlabel('Número de threads')
    plt.ylabel('Confiança')
    plt.title(f'Confiança para N = {N}')
    
    # Define os valores do eixo x para [1, 2, 4, 8]
    plt.xticks(num_threads)

    plt.savefig(f'metrics/omp/confidences/confidence_{N}.png')

    # Fecha a figura após salvá-la
    plt.close()
